In [1]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import numpy as np
import numpy as np  


def evaluate_classifier_multiple_times(classifier, X, y, n_iterations=10):  
    all_accuracies = []  
    all_f1_scores = []  
  
    for iteration in range(n_iterations):  
        # 设置十折交叉验证，每次使用不同的random_state  
        kf = KFold(n_splits=10, shuffle=True, random_state=42 + iteration * 10)  # 使用迭代次数作为随机种子  
        scores = []  
        f1_scores_iter = []  
  
        # 遍历交叉验证的每一折  
        for fold, (train_index, test_index) in enumerate(kf.split(X)):  
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]  
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]  
  
            # 将pandas读取的数据转化为list形式
            X_train = X_train.values.tolist()  
            y_train = y_train.values.tolist()  
            # X_test = X_test.values.tolist()  
            y_test = y_test.values.tolist()  
           
            X_train = np.array(X_train)
            y_train = np.array(y_train)

            nn = classifier.NeuralNetworks(X_train,y_train)
            W1, b1, W2, b2 = nn.fit(activation='relu')
            # print('训练后参数',lr.theta)
            y_pred=nn.predict(X_test, activation='relu')
            
            # one_hot索引原坐标问题，本预测要求，数据集标签从1开始，而预测的标签从0开始且连续不间断，所以预测的标签需要+1
            # Original Labels: [0 2 1 2 0]
            # One-Hot Encoded Labels:
            # [[1. 0. 0.]
            # [0. 0. 1.]
            y_pred=y_pred+1
            # print('y_pred',y_pred,len(y_pred))
            # print('y_test',y_test,len(y_test))
            accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)  
            f1 = f1_score(y_test, y_pred, average='macro')  
            scores.append(accuracy)  
            f1_scores_iter.append(f1)  
  
        mean_accuracy = np.mean(scores)  
        std_accuracy = np.std(scores)  
        mean_f1 = np.mean(f1_scores_iter)  
        print(f'第{iteration}次',mean_accuracy)  
        all_accuracies.append(mean_accuracy)  
        all_f1_scores.append(mean_f1)  
  
        # print(f"Iteration {iteration + 1}: Mean Accuracy = {mean_accuracy:.4f}, Std Accuracy = {std_accuracy:.4f}, Mean F1 Score = {mean_f1:.4f}")  

    overall_mean_accuracy = np.mean(all_accuracies)  
    overall_std_accuracy = np.std(all_accuracies)  
    overall_mean_f1 = np.mean(all_f1_scores)  
  
    return overall_mean_accuracy, overall_std_accuracy, overall_mean_f1  
  
# # 示例调用  
# # classifier_instance = YourClassifier()  # 替换为你的分类器实例  
# # X = your_X_data  # 替换为你的特征数据  
# # y = your_y_data  # 替换为你的标签数据  
# # k = your_k_value  # 替换为你的k值  
# # evaluate_classifier_multiple_times(classifier_instance, X, y, k)


In [2]:
import pandas as pd
from sklearn.discriminant_analysis import StandardScaler
from package_py import NeuralNetworks7

    
file_paths =[ "data\\hay.xls"]  # 实际文件路径
# file_paths =[ "data\\bal.xls", "data\\gla.xls", "data\\hay.xls", "data\\iri.xls", "data\\new.xls", "data\\win.xls", "data\\zoo.xls"]  # 实际文件路径
# mean_accuracys=[]
for i in range(len(file_paths)):
    file_path=file_paths[i]

    data = pd.read_excel(file_path, header=None)  
    # 将数据分为特征和标签  
    X = data.iloc[:, :-1]  # 前n列是特征  
    y = data.iloc[:, -1]   # 最后一列是分类标签  
    
    # 数据标准hua
    scaler = StandardScaler()  
    X_scaled = scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
    
    mean_accuracy,std_accuracy,f1=evaluate_classifier_multiple_times(NeuralNetworks7,X_scaled_df,y,10)
    # mean_accuracys.append(mean_accuracy)

    # 使用 f-string 格式化输出  
    print(f'{file_path} \n  mean_accuracy: {mean_accuracy:.3f} std_accuracy: {std_accuracy:.3f} f1: {f1:.3f}')



第0次 0.775
第1次 0.75
第2次 0.7625
第3次 0.71875
第4次 0.74375
第5次 0.7375
第6次 0.78125
第7次 0.775
第8次 0.75625
第9次 0.73125
data\hay.xls 
  mean_accuracy: 0.753 std_accuracy: 0.020 f1: 0.749


调库

In [15]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier  # 使用多层感知器（神经网络）
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
import numpy as np

# 文件路径列表
file_paths = [r"data\win.xls"]  # 你可以根据需要添加更多文件路径
# # 文件路径列表
# file_paths = [
#     r"data\bal.xls", r"data\gla.xls", r"data\hay.xls",
#     r"data\iri.xls", r"data\new_avoid_negtive.xls", r"data\win.xls", r"data\zoo.xls"
# ]

# 初始化结果字典
results = {}

# 对每个数据集进行十次十折交叉验证
for file_path in file_paths:
    # 读取Excel文件
    df = pd.read_excel(file_path, header=None)
    
    # 分离特征和标签
    X = df.iloc[:, :-1].values  # 将数据转换为NumPy数组，所有行，除了最后一列的所有列（特征）
    y = df.iloc[:, -1].values    # 将数据转换为NumPy数组，所有行，最后一列（标签）
    
    # 确保标签是整数类型（对于sklearn的分类器通常是必要的）
    y = y.astype(int)
    
    # 创建多层感知器分类器（神经网络）
    mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42, solver='adam', learning_rate_init=0.01)
    # hidden_layer_sizes定义了一个隐藏层，包含100个神经元；max_iter增加以避免收敛警告；solver选择'adam'进行优化
    
    # 初始化用于存储每次交叉验证结果的列表
    accuracies = []
    f1_scores_list = []
    
    # 进行十次十折交叉验证
    for i in range(10):
        kf = KFold(n_splits=10, shuffle=True, random_state=42 + i * 10)
        scores = cross_val_score(mlp, X, y, cv=kf, scoring='accuracy')
        accuracies.append(scores.mean())
        
        # 使用cross_val_predict获取所有折叠的预测
        y_preds = cross_val_predict(mlp, X, y, cv=kf)
        
        # 计算F1分数（macro平均）
        f1_scores = f1_score(y, y_preds, average='macro')
        f1_scores_list.append(f1_scores)
        print(f"数据集{file_path}第{i+1}次十折准确度: {scores.mean()}")
    
    # 计算十次交叉验证的平均准确度和F1分数
    mean_accuracy = np.mean(accuracies)
    mean_f1 = np.mean(f1_scores_list)
    print(f"数据集{file_path}平均准确度: {mean_accuracy}")
    print(f"数据集{file_path}平均F1分数: {mean_f1}")

数据集data\win.xls第1次十折准确度: 0.8258169934640522
数据集data\win.xls第2次十折准确度: 0.7065359477124183
数据集data\win.xls第3次十折准确度: 0.6767973856209151
数据集data\win.xls第4次十折准确度: 0.7575163398692809
数据集data\win.xls第5次十折准确度: 0.7509803921568627
数据集data\win.xls第6次十折准确度: 0.6594771241830066
数据集data\win.xls第7次十折准确度: 0.6954248366013072
数据集data\win.xls第8次十折准确度: 0.7284313725490196
数据集data\win.xls第9次十折准确度: 0.6186274509803923
数据集data\win.xls第10次十折准确度: 0.7859477124183007
数据集data\win.xls平均准确度: 0.7205555555555556
数据集data\win.xls平均F1分数: 0.6993545787834976


In [3]:
import numpy as np

# 激活函数及其导数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# 初始化参数
def initialize_parameters(input_dim, hidden_dim, output_dim):
    np.random.seed(42)
    W1 = np.random.randn(input_dim, hidden_dim) * np.sqrt(2. / input_dim)
    b1 = np.zeros((1, hidden_dim))
    W2 = np.random.randn(hidden_dim, output_dim) * np.sqrt(2. / hidden_dim)
    b2 = np.zeros((1, output_dim))
    return W1, b1, W2, b2

# 前向传播
def forward_propagation(X, W1, b1, W2, b2, activation='relu'):
    Z1 = np.dot(X, W1) + b1
    if activation == 'relu':
        A1 = relu(Z1)
    elif activation == 'sigmoid':
        A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)  # 输出层使用 sigmoid
    return A1, A2

# 计算损失（交叉熵损失）
def compute_loss(y_true, y_pred):
    m = y_true.shape[0]
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
   
    log_likelihood = -np.log(y_pred[y_true == 1])


    # log_likelihood = -np.log(y_pred[range(m), y_true])
    loss = np.sum(log_likelihood) / m
    return loss

# 反向传播
def backward_propagation(X, y, A1, A2, W1, W2, activation='relu'):
    m = X.shape[0]
    
    # 输出层误差
    dZ2 = A2 - y
    dW2 = (1 / m) * np.dot(A1.T, dZ2)
    db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)
    
    # 隐藏层误差
    if activation == 'relu':
        dA1 = np.dot(dZ2, W2.T) * relu_derivative(A1)
    elif activation == 'sigmoid':
        dA1 = np.dot(dZ2, W2.T) * sigmoid_derivative(A1)
    dZ1 = dA1
    dW1 = (1 / m) * np.dot(X.T, dZ1)
    db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)
    
    return dW1, db1, dW2, db2

# 更新参数
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

# 训练模型
def train(X, y, input_dim, hidden_dim, output_dim, epochs, learning_rate, activation='relu'):
    W1, b1, W2, b2 = initialize_parameters(input_dim, hidden_dim, output_dim)
    for epoch in range(epochs):
        A1, A2 = forward_propagation(X, W1, b1, W2, b2, activation)
        # print(A2.shape)
        # print(y.shape)
        loss = compute_loss(y, A2)
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss}')
        dW1, db1, dW2, db2 = backward_propagation(X, y, A1, A2, W1, W2, activation)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
    return W1, b1, W2, b2

# 预测
def predict(X, W1, b1, W2, b2, activation='relu'):
    A1, A2 = forward_propagation(X, W1, b1, W2, b2, activation)
    y_pred = np.argmax(A2, axis=1)
    return y_pred

# 生成数据
np.random.seed(42)
X = np.random.randn(100, 2)
y = (X[:, 0] + X[:, 1] > 0).astype(int)  # 简单的线性分类
y = y.reshape(-1, 1)  # 确保 y 的形状为 (n_samples, 1)

# 标准化数据
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std

# 训练模型
input_dim = 2
hidden_dim = 10
output_dim = 1
epochs = 1000
learning_rate = 0.01

W1, b1, W2, b2 = train(X, y, input_dim, hidden_dim, output_dim, epochs, learning_rate, activation='relu')

# 测试模型
y_pred = predict(X, W1, b1, W2, b2, activation='relu')
accuracy = np.mean(y_pred == y)
print(f'Accuracy: {accuracy}')

Epoch 0, Loss: 0.5351582268907642
Epoch 100, Loss: 0.2927935920643684
Epoch 200, Loss: 0.20486953535773558
Epoch 300, Loss: 0.16515315875573516
Epoch 400, Loss: 0.13943172664081677
Epoch 500, Loss: 0.12043535752130399
Epoch 600, Loss: 0.10587705457041215
Epoch 700, Loss: 0.09461338156360523
Epoch 800, Loss: 0.08538831420687877
Epoch 900, Loss: 0.07803506211895396
Accuracy: 0.56


In [1]:
import numpy as np

# 生成一些简单的二维数据
np.random.seed(42)
X = np.random.randn(100, 2)
y = (X[:, 0] + X[:, 1] > 0).astype(int)  # 简单的线性分类

# 标准化数据
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std
print(X)

[[ 7.18619012e-01 -1.73352446e-01]
 [ 8.95814650e-01  1.49821901e+00]
 [-1.39185696e-01 -2.69818200e-01]
 [ 1.98912620e+00  7.37949528e-01]
 [-4.15377295e-01  5.11683696e-01]
 [-4.08268674e-01 -5.02843704e-01]
 [ 4.19621786e-01 -1.95934920e+00]
 [-1.88886634e+00 -5.99998817e-01]
 [-1.05310431e+00  2.81958570e-01]
 [-9.30094359e-01 -1.45527346e+00]
 [ 1.85583833e+00 -2.61405822e-01]
 [ 2.14891985e-01 -1.46779493e+00]
 [-5.03295725e-01  7.73759949e-02]
 [-1.21526284e+00  3.43789409e-01]
 [-5.69322234e-01 -3.27731058e-01]
 [-5.70575634e-01  1.82950417e+00]
 [ 1.19794047e-01 -1.09848706e+00]
 [ 1.10104010e+00 -1.26262898e+00]
 [ 3.80774526e-01 -2.00602607e+00]
 [-1.42323013e+00  1.63846285e-01]
 [ 1.00235904e+00  1.38195623e-01]
 [-9.86215258e-05 -3.37199217e-01]
 [-1.59967644e+00 -7.58530192e-01]
 [-4.05007109e-01  1.02942912e+00]
 [ 5.38933436e-01 -1.80817968e+00]
 [ 5.16006392e-01 -4.21697320e-01]
 [-6.58854441e-01  5.81227516e-01]
 [ 1.34569914e+00  9.02808518e-01]
 [-8.49337475e-01 -3

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def sigmoid_derivative(x):
    return x * (1 - x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)  

In [3]:
def initialize_parameters(input_dim, hidden_dim, output_dim):
    np.random.seed(42)
    W1 = np.random.randn(input_dim, hidden_dim) * np.sqrt(2. / input_dim)
    b1 = np.zeros((1, hidden_dim))
    W2 = np.random.randn(hidden_dim, output_dim) * np.sqrt(2. / hidden_dim)
    b2 = np.zeros((1, output_dim))
    return W1, b1, W2, b2

In [4]:
def forward_propagation(X, W1, b1, W2, b2, activation='relu'):
    Z1 = np.dot(X, W1) + b1
    if activation == 'relu':
        A1 = relu(Z1)
    elif activation == 'sigmoid':
        A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)  # 输出层通常使用 sigmoid 或 softmax
    return A1, A2

In [ ]:
def compute_loss(y_true, y_pred):
    """
    计算多类别交叉熵损失函数

    参数:
    y_true (ndarray): 形状为 (m, k) 的真实标签，其中 m 是样本数量，k 是类别数量
    y_pred (ndarray): 形状为 (m, k) 的预测概率，其中 m 是样本数量，k 是类别数量

    返回:
    loss (float): 计算得到的损失值
    """
    # 获取样本数量
    m = y_true.shape[0]
    # 定义一个极小值，用于避免对数计算中的除以零错误
    epsilon = 1e-15
    # 将预测概率限制在 [epsilon, 1 - epsilon] 区间内，以避免对数计算中的无穷大
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
    # 计算每个样本的对数似然损失
    log_likelihood = -np.log(y_pred[range(m), y_true.flatten()])
    # 计算平均损失
    loss = np.sum(log_likelihood) / m
    return loss


In [6]:
def backward_propagation(X, y, A1, A2, W1, W2, activation='relu'):
    m = X.shape[0]
    
    # 输出层误差
    dZ2 = A2 - y
    dW2 = (1 / m) * np.dot(A1.T, dZ2)
    db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)
    
    # 隐藏层误差
    if activation == 'relu':
        dA1 = np.dot(dZ2, W2.T) * relu_derivative(A1)
    elif activation == 'sigmoid':
        dA1 = np.dot(dZ2, W2.T) * sigmoid_derivative(A1)
    dZ1 = dA1
    dW1 = (1 / m) * np.dot(X.T, dZ1)
    db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)
    
    return dW1, db1, dW2, db2

In [7]:
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

In [8]:
def train(X, y, input_dim, hidden_dim, output_dim, epochs, learning_rate, activation='relu'):
    W1, b1, W2, b2 = initialize_parameters(input_dim, hidden_dim, output_dim)
    for epoch in range(epochs):
        A1, A2 = forward_propagation(X, W1, b1, W2, b2, activation)
        loss = compute_loss(y, A2)
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss}')
        dW1, db1, dW2, db2 = backward_propagation(X, y, A1, A2, W1, W2, activation)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
    return W1, b1, W2, b2

In [9]:
def predict(X, W1, b1, W2, b2, activation='relu'):
    A1, A2 = forward_propagation(X, W1, b1, W2, b2, activation)
    y_pred = np.argmax(A2, axis=1)
    return y_pred

# 训练模型
input_dim = 2
hidden_dim = 10
output_dim = 1
epochs = 1000
learning_rate = 0.01

W1, b1, W2, b2 = train(X, y.reshape(-1, 1), input_dim, hidden_dim, output_dim, epochs, learning_rate, activation='relu')

# 测试模型
y_pred = predict(X, W1, b1, W2, b2, activation='relu')
accuracy = np.mean(y_pred == y)
print(f'Accuracy: {accuracy}')

IndexError: index 1 is out of bounds for axis 1 with size 1